In [27]:
import pandas as pd
from pathlib import Path
import spacy 

In [28]:
DATA_PATH="../data/raw/aclImdb_v1/aclImdb/train/"
RANDOM_SEED=42

In [29]:
columns=["Review", "Label"]
df = pd.DataFrame(columns=columns)

Add positive reviews

In [30]:
p = Path(DATA_PATH ) / 'pos'
for file_path in p.iterdir():
    with file_path.open('r') as file:
        new_row={'Review':file.read(), 'Label':1}
        df.loc[len(df)]=new_row

Add negative reviews

In [ ]:
p = Path(DATA_PATH) / 'neg'
for file_path in p.iterdir():
    with file_path.open('r') as file:
        new_raw={'Review':file.read(), 'Label':1}

In [32]:
df.head(5)

,Review,Label
0,I came in in the middle of this film so I had ...,1
1,Clint Eastwood reprises his role as Dirty Harr...,1
2,Okay this movie fine like I said but you surel...,1
3,When a small hobbit named Frodo Baggins inheri...,1
4,This film was made in 1943 when i think Judy w...,1


lowercase text

In [33]:
def lower_case(text):
    return text.lower()

df['Review'] = df['Review'].apply(lower_case)

In [34]:
df.head()

,Review,Label
0,i came in in the middle of this film so i had ...,1
1,clint eastwood reprises his role as dirty harr...,1
2,okay this movie fine like i said but you surel...,1
3,when a small hobbit named frodo baggins inheri...,1
4,this film was made in 1943 when i think judy w...,1


Tokenization and lemmatization

In [35]:
nlp=spacy.load('en_core_web_sm')

In [36]:
def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

# Apply tokenization to each row in the DataFrame
df['tokens'] = df['Review'].apply(tokenize_text)


In [37]:
df.head()

,Review,Label,tokens
0,i came in in the middle of this film so i had ...,1,"[come, middle, film, idea, credit, title, till..."
1,clint eastwood reprises his role as dirty harr...,1,"[clint, eastwood, reprise, role, dirty, harry,..."
2,okay this movie fine like i said but you surel...,1,"[okay, movie, fine, like, say, surely, need, w..."
3,when a small hobbit named frodo baggins inheri...,1,"[small, hobbit, name, frodo, baggin, inherit, ..."
4,this film was made in 1943 when i think judy w...,1,"[film, 1943, think, judy, peak, look, wise, pr..."
